In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import csv
import requests

In [2]:
url = "https://humanist.kdl.kcl.ac.uk/"
result = requests.get(url)

print(result.status_code)
print(result.headers)
page = result.content

# How would we get all the links to volumes from the Humanist Page?

soup = BeautifulSoup(page)

200
{'Server': 'ATS', 'Date': 'Wed, 23 Jun 2021 16:00:40 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '5814', 'X-Frame-Options': 'SAMEORIGIN', 'Age': '0', 'Connection': 'keep-alive'}


https://humanist.kdl.kcl.ac.uk/Archives/Current/ https://humanist.kdl.kcl.ac.uk/Archives/Converted_Text/

In [3]:
links = ['https://humanist.kdl.kcl.ac.uk/Archives/Current/','https://humanist.kdl.kcl.ac.uk/Archives/Converted_Text/']
texts = []
for link in links:
    result = requests.get(link)
    page = result.content

    soup = BeautifulSoup(page)
    text_links = soup.find_all('a')
    for txt_link in text_links:
        url = txt_link.get('href')
        if 'txt' in url:
            print(link+url)
            text_result = requests.get(link+url)
            text_page = text_result.content
            text_soup = BeautifulSoup(text_page)
            dates = url.split('.')[1]
            text = text_soup.get_text()
            texts.append({'dates': dates, 'text': text})

https://humanist.kdl.kcl.ac.uk/Archives/Current/Humanist.21.542-21.680.txt
https://humanist.kdl.kcl.ac.uk/Archives/Current/Humanist.vol22.txt


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


https://humanist.kdl.kcl.ac.uk/Archives/Current/Humanist.vol23.txt
https://humanist.kdl.kcl.ac.uk/Archives/Current/Humanist.vol24.txt
https://humanist.kdl.kcl.ac.uk/Archives/Current/Humanist.vol25.txt
https://humanist.kdl.kcl.ac.uk/Archives/Current/Humanist.vol26.txt


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


https://humanist.kdl.kcl.ac.uk/Archives/Current/Humanist.vol27.txt


...

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


https://humanist.kdl.kcl.ac.uk/Archives/Converted_Text/humanist.2004-2005.txt
https://humanist.kdl.kcl.ac.uk/Archives/Converted_Text/humanist.2005-2006.txt


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


https://humanist.kdl.kcl.ac.uk/Archives/Converted_Text/humanist.2006-2007.txt
https://humanist.kdl.kcl.ac.uk/Archives/Converted_Text/humanist.2007-2008.txt


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [6]:
# # Turn our web scraped data in a dataframe
humanist_vols = pd.DataFrame.from_dict(texts, orient='columns')

In [10]:
# Write to csv
humanist_vols.to_csv('web_scraped_humanist_listserv.csv')